In [1]:
# -*- coding: utf-8 -*-
"""This module defines functions for Dali searching Protein Data Bank."""

import re
import numpy as np
from prody.atomic import Atomic, AtomGroup, AtomMap
from prody.proteins.pdbfile import _getPDBid
from prody.measure import getRMSD, getTransformation
from prody.utilities import checkCoords, checkWeights, createStringIO
from prody import LOGGER, PY3K
from prody import parsePDB, writePDBStream
# if PY3K:
    # import urllib.parse as urllib
    # import urllib.request as urllib2
# else:
    # import urllib
    # import urllib2
from prody.ensemble import Ensemble
from prody.ensemble import PDBEnsemble
import os

__all__ = ['DaliRecord', 'searchDali', 
           'daliFilterMultimer', 'daliFilterMultimers']

def searchDal(pdb, chain=None, subset='fullPDB', daliURL=None, **kwargs):
    """Search Dali server with input of PDB ID (or local PDB file) and chain ID.
    Dali server: http://ekhidna2.biocenter.helsinki.fi/dali/
    
    :arg pdb: PDB code or local PDB file for the protein to be searched

    :arg chain: chain identifier (only one chain can be assigned for PDB)
    :type chain: str

    :arg subset: fullPDB, PDB25, PDB50, PDB90
    :type subset: str
    """
    
    import requests
    
    LOGGER.timeit('_dali')
    # timeout = 120
    timeout = kwargs.pop('timeout', 120)
    
    if daliURL is None:
        daliURL = "http://ekhidna2.biocenter.helsinki.fi/cgi-bin/sans/dump.cgi"
    
    if isinstance(pdb, Atomic):
        atoms = pdb
        chain_set = set(atoms.getChids())
        if chain and not chain in chain_set:
            raise ValueError('input structure (%s) does not have chain %s'%(atoms.getTitle(), chain))
        
        if len(chain_set) > 1:
            if not chain:
                raise TypeError('the structure (%s) contains more than one chain, therefore a chain identifier '
                                'needs to be specified'%pdb.getTitle())
            atoms = atoms.select('chain '+chain)
        else:
            chain = chain_set.pop()
            
        stream = createStringIO()
        writePDBStream(stream, atoms)
        data = stream.getvalue()
        stream.close()
        files = {"file1" : data}

        pdbId = atoms.getTitle()
        pdb_chain = ''
        dali_title = 'Title_'+pdbId+chain
    elif isinstance(pdb, str):
        if os.path.isfile(pdb):
            atoms = parsePDB(pdb)
            chain_set = set(atoms.getChids())
            # pdbId = "s001"
            filename = os.path.basename(pdb)
            filename, ext = os.path.splitext(filename)
            if ext.lower() == '.gz':
                filename2, ext2 = os.path.splitext(filename)
                if ext2.lower() == '.pdb':
                    filename = filename2
            pdbId = filename
            if chain and not chain in chain_set:
                raise ValueError('input PDB file does not have chain ' + chain)
            
            if len(chain_set) > 1:
                if not chain:
                    raise TypeError('PDB file (%s) contains more than one chain, therefore a chain identifier '
                                    'needs to be specified'%pdb)
                atoms = atoms.select('chain '+chain)
                #local_temp_pdb = pdbId+chain+'.pdb'
                #local_temp_pdb = 's001'+chain+'.pdb'
                stream = createStringIO()
                writePDBStream(stream, atoms)
                data = stream.getvalue()
                stream.close()
            else:
                data = open(pdb, "rb")
                chain = chain_set.pop()
            files = {"file1" : data}
            # case: multiple chains.             apply fetch ? multiple times?
            pdb_chain = ''
            dali_title = 'Title_' + pdbId + chain
        else:
            pdbId, ch = _getPDBid(pdb)
            if not chain:
                chain = ch
            if not chain:
                raise TypeError('a chain identifier is needed for the search')
            pdb_chain = pdbId + chain
            dali_title = 'Title_' + pdb_chain
            files = ''
    parameters = { 'cd1' : pdb_chain, 'method': 'search', 'title': dali_title, 'address': '' }
    # enc_params = urllib.urlencode(parameters).encode('utf-8')
    # request = urllib2.Request(daliURL, enc_params)
    request = requests.post(daliURL, parameters, files=files)
    try_error = 3
    while try_error >= 0:
        try:
            # url = urllib2.urlopen(request).url
            url = request.url
            break
        except:
            try_error -= 1
            if try_error >= 0:
                LOGGER.sleep(2, '. Connection error happened. Trying to reconnect...')
                continue
            else:
                # url = urllib2.urlopen(request).url
                url = request.url
                break
    if url.split('.')[-1].lower() in ['html', 'php']:
        # print('test -1: '+url)
        url = url.replace(url.split('/')[-1], '')
    LOGGER.debug('Submitted Dali search for PDB "{0}{1}".'.format(pdbId, chain))
    LOGGER.info(url)
    LOGGER.clear()
    
    return DaliRecor(url, pdbId, chain, subset=subset, timeout=timeout, **kwargs)
    

class DaliRecor(object):

    """A class to store results from Dali PDB search."""

    def __init__(self, url, pdbId, chain, subset='fullPDB', localFile=False, **kwargs):
        """Instantiate a DaliRecord object instance.

        :arg url: url of Dali results page or local dali results file
        :arg pdbId: PDB code for searched protein
        :arg chain: chain identifier (only one chain can be assigned for PDB)
        :arg subset: fullPDB, PDB25, PDB50, PDB90. Ignored if localFile=True (url is a local file)
        :arg localFile: whether provided url is a path for a local dali results file
        """

        self._url = url
        self._pdbId = pdbId
        self._chain = chain
        subset = subset.upper()
        if subset == "FULLPDB" or subset not in ["PDB25", "PDB50", "PDB90"]:
            self._subset = ""
        else:
            self._subset = "-"+subset[3:]
        timeout = kwargs.pop('timeout', 120)

        self._title = pdbId + '-' + chain
        self._alignPDB = None
        self._filterDict = None
        self._max_index = None
        self.fetch(self._url, localFile=localFile, timeout=timeout, **kwargs)

    def fetch(self, url=None, localFile=False, **kwargs):
        """Get Dali record from url or file.

        :arg url: url of Dali results page or local dali results file
            If None then the url already associated with the DaliRecord object is used.
        :type url: str

        :arg localFile: whether provided url is a path for a local dali results file
        :type localFile: bool

        :arg timeout: amount of time until the query times out in seconds
            default value is 120
        :type timeout: int

        :arg localfolder: folder in which to find the local file
            default is the current folder
        :type localfolder: str
        """
        if localFile:
            dali_file = open(url, 'r')
            data = dali_file.read()
            dali_file.close()
        else:
            import requests
            
            if url == None:
                url = self._url
            
            sleep = 2
            timeout = kwargs.pop('timeout', 120)
            LOGGER.timeit('_dali')
            log_message = ''
            try_error = 3
            while True:
                LOGGER.write('Connecting to Dali for search results...')
                LOGGER.clear()
                try:
                    # html = urllib2.urlopen(url).read()
                    html = requests.get(url).content
                except:
                    try_error -= 1
                    if try_error >= 0:
                        LOGGER.sleep(2, '. Connection error happened. Trying to reconnect...')
                        continue
                    else:
                        # html = urllib2.urlopen(url).read()
                        html = requests.get(url).content
                if PY3K:
                    html = html.decode()
                if html.find('Status: Queued') > -1:
                    log_message = '(Dali search is queued)...'
                elif html.find('Status: Running') > -1:
                    log_message = '(Dali search is running)...'
                elif html.find('Your job') == -1 and html.find('.txt') > -1:
                    break
                elif html.find('ERROR:') > -1:
                    LOGGER.warn(': Dali search reported an ERROR!')
                    self.isSuccess = False
                    return False
                sleep = 20 if int(sleep * 1.5) >= 20 else int(sleep * 1.5)
                if LOGGER.timing('_dali') > timeout:
                    LOGGER.warn(': Dali search has timed out. \nThe results can be obtained later using the fetch() method.')
                    self.isSuccess = False
                    return False
                LOGGER.sleep(int(sleep), 'to reconnect to Dali '+log_message)
                LOGGER.clear()
            LOGGER.clear()
            LOGGER.report('Dali results were fetched in %.1fs.', '_dali')
            lines = html.strip().split('\n')
            file_name = re.search('=.+-90\\.txt', html).group()[1:]
            file_name = file_name[:-7]
            # LOGGER.info(url+file_name+self._subset+'.txt')
            # data = urllib2.urlopen(url+file_name+self._subset+'.txt').read()
            data = requests.get(url+file_name+self._subset+'.txt').content
            if PY3K:
                data = data.decode()
            localfolder = kwargs.pop('localfolder', '.')

            if file_name.lower().startswith('s001'):
                temp_name = self._pdbId + self._chain
            else:
                temp_name = file_name
            temp_name += self._subset + '_dali.txt'
            if localfolder != '.' and not os.path.exists(localfolder):
                os.mkdir(localfolder)
            with open(localfolder+os.sep+temp_name, "w") as file_temp: file_temp.write(html + '\n' + url+file_name+self._subset+'.txt' + '\n' + data)
            # with open(temp_name, "a+") as file_temp: file_temp.write(url+file_name + '\n' + data)
        data_list = data.strip().split('# ')
        # No:  Chain   Z    rmsd lali nres  %id PDB  Description -> data_list[3]
        # Structural equivalences -> data_list[4]
        # Translation-rotation matrices -> data_list[5]
        map_temp_dict = dict()
        lines = data_list[4].strip().split('\n')
        self._lines_4 = lines
        mapping_temp = np.genfromtxt(lines[1:], delimiter = (4,1,14,6,2,4,4,5,2,4,4,3,5,4,3,5,6,3,5,4,3,5,28), 
                                     usecols = [0,3,5,7,9,12,15,15,18,21], dtype='|i4')
        # [0,3,5,7,9,12,15,15,18,21] -> [index, residue_a, residue_b, residue_i_a, residue_i_b, resid_a, resid_b, resid_i_a, resid_i_b]
        for map_i in mapping_temp:
            if not map_i[0] in map_temp_dict:
                map_temp_dict[map_i[0]] = [[map_i[1], map_i[2], map_i[3], map_i[4]]]
            else:
                map_temp_dict[map_i[0]].append([map_i[1], map_i[2], map_i[3], map_i[4]])
        self._max_index = max(mapping_temp[:,2])
        self._mapping = map_temp_dict
        self._data = data_list[3]
        lines = data_list[3].strip().split('\n')
        # daliInfo = np.genfromtxt(lines[1:], delimiter = (4,3,6,5,5,5,6,5,57), usecols = [0,2,3,4,5,6,7,8], 
                                # dtype=[('id', '<i4'), ('pdb_chain', '|S6'), ('Z', '<f4'), ('rmsd', '<f4'), 
                                # ('len_align', '<i4'), ('nres', '<i4'), ('identity', '<i4'), ('title', '|S70')])
        daliInfo = np.genfromtxt(lines[1:], delimiter = (4,3,6,5,5,5,6,5,57), usecols = [0,2,3,4,5,6,7,8], 
                                dtype=[('id', '<i4'), ('pdb_chain', '|U6'), ('Z', '<f4'), ('rmsd', '<f4'), 
                                ('len_align', '<i4'), ('nres', '<i4'), ('identity', '<i4'), ('title', '|U70')])
        if daliInfo.ndim == 0:
            daliInfo = np.array([daliInfo])
        pdbListAll = []
        self._daliInfo = daliInfo
        dali_temp_dict = dict()
        for temp in self._daliInfo:
            temp_dict = dict()
            pdb_chain = temp[1].strip()[0:6]
            # U6 and U70 were used as the dtype for np.genfromtext -> unicode string were used in daliInfo 
            # if PY3K:
                # pdb_chain = pdb_chain.decode()
            pdb_chain = str(pdb_chain)
            temp_dict['pdbId'] = pdbid = pdb_chain[0:4].lower()
            temp_dict['chainId'] = chid = pdb_chain[5:6]
            temp_dict['pdb_chain'] = pdb_chain = pdbid + chid
            temp_dict['Z'] = temp[2]
            temp_dict['rmsd'] = temp[3]
            temp_dict['len_align'] = temp[4]
            temp_dict['nres'] = temp[5]
            temp_dict['identity'] = temp[6]
            temp_dict['mapping'] = (np.array(map_temp_dict[temp[0]])-1).tolist()
            temp_dict['map_ref'] = [x for map_i in (np.array(map_temp_dict[temp[0]])-1).tolist() for x in range(map_i[0], map_i[1]+1)]
            temp_dict['map_sel'] = [x for map_i in (np.array(map_temp_dict[temp[0]])-1).tolist() for x in range(map_i[2], map_i[3]+1)]
            dali_temp_dict[pdb_chain] = temp_dict
            pdbListAll.append(pdb_chain)
        self._pdbListAll = tuple(pdbListAll)
        self._pdbList = self._pdbListAll
        self._alignPDB = dali_temp_dict
        LOGGER.info('Obtained ' + str(len(pdbListAll)) + ' PDB chains from Dali for '+self._pdbId+self._chain+'.')
        self.isSuccess = True
        return True
        
    def getPDBs(self, filtered=True):
        """Returns PDB list (filters may be applied)"""

        if self._alignPDB is None:
            LOGGER.warn('Dali Record does not have any data yet. Please run fetch.')
        
        if filtered:
            return self._pdbList
        return self._pdbListAll
        
    def getHits(self):
        """Returns the dictionary associated with the DaliRecord"""

        if self._alignPDB is None:
            LOGGER.warn('Dali Record does not have any data yet. Please run fetch.')

        return self._alignPDB
        
    def getFilterList(self):
        """Returns a list of PDB IDs and chains for the entries that were filtered out"""
        
        filterDict = self._filterDict
        if filterDict is None:
            raise ValueError('You cannot obtain the list of filtered out entries before doing any filtering.')

        temp_str = ', '.join([str(len(filterDict['len'])), str(len(filterDict['rmsd'])), 
                            str(len(filterDict['Z'])), str(len(filterDict['identity']))])
        LOGGER.info('Filtered out [' + temp_str + '] for [length, RMSD, Z, identity]')
        return self._filterList

    
    def getMapping(self, key):
        """Get mapping for a particular entry in the DaliRecord"""

        if self._alignPDB is None:
            LOGGER.warn('Dali Record does not have any data yet. Please run fetch.')
            return None
        
        try:
            info = self._alignPDB[key]
            mapping = [info['map_ref'], info['map_sel']]
        except KeyError:
            return None
        return mapping

    def getMappings(self):
        """Get all mappings in the DaliRecord"""

        if self._alignPDB is None:
            LOGGER.warn('Dali Record does not have any data yet. Please run fetch.')
            return None

        map_dict = {}
        for key in self._alignPDB:
            info = self._alignPDB[key]
            mapping = [info['map_ref'], info['map_sel']]
            map_dict[key] = mapping
        return map_dict

    mappings = property(getMappings)

    def filter(self, cutoff_len=None, cutoff_rmsd=None, cutoff_Z=None, cutoff_identity=None):
        """Filters out PDBs from the PDBList and returns the PDB list.
        PDBs that satisfy any of the following criterion will be filtered out.
        (1) Length of aligned residues < cutoff_len (must be an integer or a float between 0 and 1);
        (2) RMSD < cutoff_rmsd (must be a positive number);
        (3) Z score < cutoff_Z (must be a positive number);
        (4) Identity > cutoff_identity (must be an integer or a float between 0 and 1).
        """
        if self._max_index is None:
            LOGGER.warn('DaliRecord has no data. Please use the fetch() method.')
            return None

        if cutoff_len == None:
            # cutoff_len = int(0.8*self._max_index)
            cutoff_len = 0.00000001
        elif not isinstance(cutoff_len, (float, int)):
            raise TypeError('cutoff_len must be a float or an integer')
        elif cutoff_len <= 1 and cutoff_len > 0:
            cutoff_len = int(cutoff_len*self._max_index)
        elif cutoff_len <= self._max_index and cutoff_len > 0:
            cutoff_len = int(cutoff_len)
        else:
            raise ValueError('cutoff_len must be a float between 0 and 1, or an int not greater than the max length')
            
        if cutoff_rmsd == None:
            cutoff_rmsd = 1000
        elif not isinstance(cutoff_rmsd, (float, int)):
            raise TypeError('cutoff_rmsd must be a float or an integer')
        elif cutoff_rmsd >= 0:
            cutoff_rmsd = float(cutoff_rmsd)
        else:
            raise ValueError('cutoff_rmsd must be a number not less than 0')
            
        if cutoff_Z == None:
            cutoff_Z = 0
        elif not isinstance(cutoff_Z, (float, int)):
            raise TypeError('cutoff_Z must be a float or an integer')
        elif cutoff_Z >= 0:
            cutoff_Z = float(cutoff_Z)
        else:
            raise ValueError('cutoff_Z must be a number not less than 0')
            
        if cutoff_identity == None or cutoff_identity == 0:
            cutoff_identity = 0
        elif not isinstance(cutoff_identity, (float, int)):
            raise TypeError('cutoff_identity must be a float or an integer')
        elif cutoff_identity <= 1 and cutoff_identity > 0:
            cutoff_identity = float(cutoff_identity*100)
        elif cutoff_identity <= 100 and cutoff_identity > 0:
            cutoff_identity = float(cutoff_identity)
        else:
            raise ValueError('cutoff_identity must be a float between 0 and 1, or a number between 0 and 100')
            
        # debug:
        # print('cutoff_len: ' + str(cutoff_len) + ', ' + 'cutoff_rmsd: ' + str(cutoff_rmsd) + ', ' + 'cutoff_Z: ' + str(cutoff_Z) + ', ' + 'cutoff_identity: ' + str(cutoff_identity))
        
        daliInfo = self._alignPDB
        if daliInfo is None:
            raise ValueError("Dali Record does not have any data yet. Please run fetch.")

        pdbListAll = self._pdbListAll
        missing_ind_dict = dict()
        ref_indices_set = set(range(self._max_index))
        filterListLen = []
        filterListRMSD = []
        filterListZ = []
        filterListIdentity = []
        
        RMSDs = []
        
        # keep the first PDB (query PDB)
        for pdb_chain in pdbListAll[1:]:
            temp_dict = daliInfo[pdb_chain]
            # print ('currRMSD', temp_dict['rmsd'])
            # filter: len_align, identity, rmsd, Z
            RMSDs.append(temp_dict['rmsd'])
            if temp_dict['len_align'] < cutoff_len:
                # print('Filter out ' + pdb_chain + ', len_align: ' + str(temp_dict['len_align']))
                filterListLen.append(pdb_chain)
                continue
            if temp_dict['rmsd'] > cutoff_rmsd:
                # print('Filter out ' + pdb_chain + ', rmsd: ' + str(temp_dict['rmsd']))
                filterListRMSD.append(pdb_chain)
                # print ('currRMSD', temp_dict['rmsd'])
                # print(pdb_chain)
                continue
            if temp_dict['Z'] < cutoff_Z:
                # print('Filter out ' + pdb_chain + ', Z: ' + str(temp_dict['Z']))
                filterListZ.append(pdb_chain)
                continue
            if temp_dict['identity'] < cutoff_identity:
                # print('Filter out ' + pdb_chain + ', identity: ' + str(temp_dict['identity']))
                filterListIdentity.append(pdb_chain)
                continue
            temp_diff = list(ref_indices_set - set(temp_dict['map_ref']))
            for diff_i in temp_diff:
                if not diff_i in missing_ind_dict:
                    missing_ind_dict[diff_i] = 1
                else:
                    missing_ind_dict[diff_i] += 1
        self._missing_ind_dict = missing_ind_dict
        
        filterList = filterListLen + filterListRMSD + filterListZ + filterListIdentity
        filterDict = {'len': filterListLen, 'rmsd': filterListRMSD, 'Z': filterListZ, 'identity': filterListIdentity}
        self._filterList = filterList
        self._filterDict = filterDict
        self._pdbList = [self._pdbListAll[0]] + [item for item in self._pdbListAll[1:] if not item in filterList]
        LOGGER.info(str(len(filterList)) + ' PDBs have been filtered out from '+str(len(pdbListAll))+' Dali hits (remaining: '+str(len(pdbListAll)-len(filterList))+').')
        
        filterRMSD = []
        diverseRMSD = []
        for item in RMSDs:
            if item < cutoff_rmsd:
                filterRMSD.append(item)
            if item > 1.0:
                diverseRMSD.append(item)
        
        print ('RMSD less than ', cutoff_rmsd, mean(filterRMSD), std(filterRMSD))
        print ('RMSD greater than 1A ', mean(diverseRMSD), std(diverseRMSD))
        
        return self._pdbList
    
    def getTitle(self):
        """Return the title of the record"""

        return self._title

def daliFilterMultimer(atoms, dali_rec, n_chains=None):
    """
    Filters multimers to only include chains with Dali mappings.

    :arg atoms: the multimer to be filtered
    :type atoms: :class:`.Atomic`

    :arg dali_rec: the DaliRecord object with which to filter chains
    :type dali_rec: :class:`.DaliRecord`
    """
    if not isinstance(atoms, Atomic):
        raise TypeError("atoms should be an Atomic object")

    if not isinstance(dali_rec, DaliRecord):
        raise TypeError("dali_rec should be a DaliRecord")
    try:
        keys = dali_rec._alignPDB
    except:
        raise AttributeError("Dali Record does not have any data yet. Please run fetch.")

    numChains = 0
    atommap = None
    for i, chain in enumerate(atoms.iterChains()):
        m = dali_rec.getMapping(chain.getTitle()[:4] + chain.getChid())
        if m is not None:
            numChains += 1
            if atommap is None:
                atommap = chain
            else:
                atommap += chain

    if n_chains is None or numChains == n_chains:
        return atommap
    else:
        return None

def daliFilterMultimers(structures, dali_rec, n_chains=None):
    """A wrapper for daliFilterMultimer to apply to multiple structures.
    """
    dali_ags = []
    for entry in structures:
        result = daliFilterMultimer(entry, dali_rec, n_chains)
        if result is not None:
            dali_ags.append(result)
    return dali_ags

In [2]:
from numpy import *
from matplotlib.pyplot import *
# import os
ion()

# find hinges
def splitHinge(vals, sites, block):
    # split hinges
    currBlock = 1
    index = 0
    isStart = False
    newSites = []
    newVals = []

    for val in vals:
        if block[index] == currBlock and not isStart:
            isStart = True
            currSites = [sites[index]]
            currValLists = [vals[index]]
        elif block[index] == currBlock and isStart:
            currSites.append(sites[index])
            currValLists.append(vals[index])
        else:
            currBlock += 1
            isStart = True
            newSites.append(currSites)
            newVals.append(currValLists)
            currSites = [sites[index]]
            currValLists = [vals[index]]
        if currBlock == block[-1] and index == len(vals) - 1:
            newSites.append(currSites)
            newVals.append(currValLists)
        index += 1
    # print newSites, newVals
    return newSites, newVals


def mini(a, b):
    if a > b:
        return a
    else:
        return b


def checkIfCrossBase(currVal, currSite, currEigenVec):
    identify = sort(currVal)
    ifRemain = True
    if identify[0] < 0 and identify[-1] > 0:
        ifRemain = True
    else:
        maxSite = max(currSite)
        minSite = min(currSite)
        # print (identify)
        if identify[0] >= 0 and identify[-1] >= 0:
            if currEigenVec[max(0, minSite - 1)] < 0 or currEigenVec[min(len(currEigenVec) - 1, maxSite + 1)] < 0:
                ifRemain = True
            elif currEigenVec[max(0, minSite - 2)] < 0 or currEigenVec[min(len(currEigenVec) - 1, maxSite + 2)] < 0:
                ifRemain = True
            elif currEigenVec[max(0, minSite - 3)] < 0 or currEigenVec[min(len(currEigenVec) - 1, maxSite + 3)] < 0:
                ifRemain = True
            else:
                ifRemain = False
        elif identify[0] <= 0 and identify[-1] <= 0:
            # print (len(currEigenVec))
            # print (currEigenVec[239])

            if currEigenVec[max(0, minSite - 1)] > 0 or currEigenVec[min(len(currEigenVec) - 1, (maxSite + 1))] > 0:
                ifRemain = True
            elif currEigenVec[max(0, minSite - 2)] > 0 or currEigenVec[min(len(currEigenVec) - 1, (maxSite + 2))] > 0:
                ifRemain = True
            elif currEigenVec[max(0, minSite - 3)] > 0 or currEigenVec[min(len(currEigenVec) - 1, (maxSite + 3))] > 0:
                ifRemain = True
            else:
                ifRemain = False

        else:
            print(identify[0], identify[-1])
            print("Check!")
    return ifRemain


def oneModeHinge(currEigenVec, thereshold=20):
    range = ((1 / float(len(currEigenVec))) ** 0.5) / thereshold
    # print (range)
    # print range
    sites = []
    vals = []
    block = []

    index = 0
    blockID = 1
    for item in currEigenVec:
        if item <= range and item >= -range:
            vals.append(item)
            if len(sites) >= 1 and abs(sites[-1] - index) > 1:
                blockID += 1
            sites.append(index)
            block.append(blockID)
        index += 1

    newSites, newVals = splitHinge(vals, sites, block)

    filtSites = []
    filtVals = []

    index = 0
    for item in newSites:
        currSite = newSites[index]
        currVal = newVals[index]
        if checkIfCrossBase(currVal, currSite, currEigenVec):
            filtSites.append(currSite)
            filtVals.append(currVal)
        index += 1

    return filtSites, filtVals


def getValue(currList):
    newList = []
    for item in currList:
        newList.append(float(item))
    return newList


def readModes(fileName):
    allModes = {}
    with open(fileName, 'r') as rf:
        for line in rf.readlines():
            line = line.strip()
            line = line.split('\t')
            currMode = line[0]
            currVal = line[1:]
            allModes[currMode] = getValue(currVal)
    return allModes


def flattenData(arrayTwoD):
    flat = []
    for item in arrayTwoD:
        for each in item:
            flat.append(each)
    return flat

def cleanStr(currStr):
    return currStr.strip('\'')

def readFile(fileName):
    data = {}
    with open(fileName, 'r') as rf:
        for line in rf.readlines():
            line = line.replace('\n', '')
            line = line.strip()
            line = line.split('\t')
            currPDB = line[0]
            currDrug = line[1]
            currChain = line[2:]
            if currPDB not in data.keys():
                data[currPDB] = {}
            if currDrug not in data[currPDB].keys():
                data[currPDB][currDrug] = []
            for each in currChain:
                data[currPDB][currDrug].append(cleanStr(each))
    return data

# contribution of mode of motions
def getContribution(eigenVals, mode):
    allContribution = np.sqrt(np.sum(1 / eigenVals))
    currModeContribution = eigenVals[:mode]
    currContribution = np.sqrt(np.sum(1 / currModeContribution))
    return currContribution / allContribution

def getModesGivenThreshold(eigenVals, thereshold):
    contribution = 0
    mode = 0
    while contribution <= thereshold:
        mode += 1
        contribution = getContribution(eigenVals, mode)
    return mode

def writeBinding(fileName, pdb_ids):
    with open(fileName, 'w') as wf:
        for item in pdb_ids:
            wf.write(item[:4] + '\t' + item[-1])
            wf.write('\n')


from prody import *
def getModes(currPDB, eachChain, length = 0.7, rmsd = 1.0, Z = 10):    
    # if not os.path.exists('./Results/' + currPath):
    #     os.mkdir('./Results/' + currPath)
    dali_rec = searchDal(currPDB, eachChain)
    while not dali_rec.isSuccess:
        dali_rec.fetch()

    pdb_ids = dali_rec.filter(cutoff_len = length, cutoff_rmsd = rmsd, cutoff_Z = Z)
    # pdb_ids = dali_rec.filter(cutoff_identity=0.5)

    currNumber = len(pdb_ids) # this number should over 50 as cutoffs

    print ('# of similar structures is found from Dali', currNumber)
    mappings = dali_rec.getMappings()
    
    ags = parsePDB(pdb_ids, subset='ca')
    dali_ens = buildPDBEnsemble(ags, mapping=mappings, seqid=20)
    gnms = calcEnsembleENMs(dali_ens, model='GNM', trim='reduce', n_modes=None)
    eigVals = gnms.getEigvals()
    averageEigVals = mean(eigVals, axis=0)

    eigVects = gnms.getEigvecs()
    averageEigVects = mean(eigVects, axis=0)
    return averageEigVals, averageEigVects, pdb_ids, gnms, averageEigVects

            
def getDali_info(currPDB, eachChain, rmsd = 2.5, length = 0.9):
    dali_rec = searchDali(currPDB, eachChain)
    while not dali_rec.isSuccess:
        dali_rec.fetch()

    pdb_ids = dali_rec.filter(cutoff_len=length, cutoff_rmsd=rmsd)
    # pdb_ids = dali_rec.filter(cutoff_identity=0.5)

    currNumber = len(pdb_ids) # this number should over 50 as cutoffs

    print ('# of similar structures is found from Dali', currNumber)
    mappings = dali_rec.getMappings()
    return pdb_ids, mappings

def getHinges(averageEigVects, currNumModes, bandWidth = 15):
    currHinge = []
    for i in range(currNumModes):
        # print (i, len(averageEigVects[:, i]))
        currPos, currVals = oneModeHinge(averageEigVects[:, i], thereshold=bandWidth)
        currPos = flattenData(currPos)
        # print (currPos)
        currHinge += currPos
        currHinge = list(set(currHinge))
    return currHinge


def getHinges_modes(averageEigVects, mode, bandWidth = 15):
    currHinge = []
    # print (i, len(averageEigVects[:, i]))
    currPos, currVals = oneModeHinge(averageEigVects[:, mode], thereshold=bandWidth)
    currPos = flattenData(currPos)
        # print (currPos)
    currHinge += currPos
    currHinge = list(set(currHinge))
    return currHinge

def get3Hinges_bandwidth(averageEigVects, bandWidths = [15, 16, 17]):
    currHinge = []
    for i in range(3):
        # print (i, len(averageEigVects[:, i]))
        currPos, currVals = oneModeHinge(averageEigVects[:, i], thereshold=bandWidths[i])
        currPos = flattenData(currPos)
        # print (currPos)
        currHinge += currPos
        currHinge = list(set(currHinge))
    return currHinge

In [3]:
# Experiment to evaluate performance of hypergeometric p values
from scipy.stats import hypergeom
def ORA(M, N, n, k):
    total = 0
    totalLength = N
    binding = M
    hinge = n
    overlap = k
    
    for i in range(overlap):
        total += hypergeom.pmf(i, totalLength, binding, hinge)
    return 1 - total

import os
def writeIDs(path, idList):
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:

       # Create a new directory because it does not exist
       os.makedirs(path)
       print("The new directory is created!")
    
    with open(path + '/ids.txt', 'w') as wf:
        for item in idList:
            wf.write(item[:4] + '\t' + item[-1])
            wf.write('\n')

In [4]:
def screenIDs(ids):
    newIDs = []
    for item in ids:
        newIDs.append(item[:4])
    return newIDs

# filter transmembrane protein
def filterHinges(Hinges_2, ContinueThreshold = 50, HingeCutoff = 10):
    newHinges = []
    currItem = []
    for item in Hinges_2:
        if currItem == [] or currItem[-1] == item - 1:
            currItem.append(item)
        else:
            newHinges.append(currItem)
            currItem = []

    filteredItem = []
    for item in newHinges:
        if len(item) <= ContinueThreshold:
            filteredItem += item
        else:
            filteredItem += item[:HingeCutoff]
            filteredItem += item[-HingeCutoff:]
    return filteredItem 

# Get ensemble

In [8]:
currPDB = '1ra3'
eachChain = 'A'
averageEigVals, averageEigVects, ids, gnms, averageEigVects = getModes(currPDB, eachChain, length=0.8, rmsd=2, Z=10)

@> Submitted Dali search for PDB "1ra3A".
@> http://ekhidna2.biocenter.helsinki.fi/barcosel/tmp//1ra3A/
@> Dali results were fetched in 0.2s.   
@> Obtained 1217 PDB chains from Dali for 1ra3A.
@> 262 PDBs have been filtered out from 1217 Dali hits (remaining: 955).
@> Retrieving 1ra3A... [  0%]

RMSD less than  2.0 1.4319226 0.45202386
RMSD greater than 1A  1.9801792 0.57505894
# of similar structures is found from Dali 955


@> 955 PDBs were parsed in 263.90s.
@> Mapping 4kjlA_ca to the reference... [ 17%] 11s@> WARNING 
the number of residues in the map_dict (161 residues) is inconsistent with Chain A from 4kjlA_ca (157 residues)
@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6rumA_ca to the reference... [ 54%] 7s @> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6rulA_ca to the reference... [ 55%] 7s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6aohB_ca to the reference... [ 56%] 6s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6n1tB_ca to the reference... [ 56%] 6s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 4eilE_ca to the reference... [ 56%] 6s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 4kyaG_ca to the reference... [ 56%] 6s@> WARNING no atommaps were avail

@> Mapping 6lhjA_ca to the reference... [ 63%] 6s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6pfeA_ca to the reference... [ 63%] 5s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6pfaD_ca to the reference... [ 63%] 5s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 4ky8E_ca to the reference... [ 63%] 5s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 2oipC_ca to the reference... [ 63%] 5s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 4ky4A_ca to the reference... [ 64%] 5s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6n1sA_ca to the reference... [ 64%] 5s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6pfhE_ca to the reference... [ 64%] 5s@> WARNING no atommaps were available. Consider adjusting accepting c

@> Mapping 8jfbB_ca to the reference... [ 78%] 4s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 3dl6A_ca to the reference... [ 79%] 3s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6pf9E_ca to the reference... [ 79%] 3s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 8jfdB_ca to the reference... [ 80%] 3s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6pfbE_ca to the reference... [ 80%] 3s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 1qzfD_ca to the reference... [ 80%] 3s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 6kp7B_ca to the reference... [ 80%] 3s@> WARNING no atommaps were available. Consider adjusting accepting criteria
@> Mapping 3dl6C_ca to the reference... [ 81%] 3s@> WARNING no atommaps were available. Consider adjusting accepting c

...

@> WARNING last 6 modes for 144 modes from MaskedGNM 1dg5A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 6vseB_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 7 modes for 144 modes from MaskedGNM 6vsdB_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 5 modes for 144 modes from MaskedGNM 7mylD_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 7k62B_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 6nniA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 6vseA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WAR

@> WARNING last 7 modes for 144 modes from MaskedGNM 4kl9A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 5sd4A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 7 modes for 144 modes from MaskedGNM 4kneA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 5sd5A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 5sd2A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 5sd1A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 5scxA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WAR

@> WARNING last 10 modes for 144 modes from MaskedGNM 3ghwA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 6de4A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 3ghvA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 3nxoA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 4kd7B_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 1s3wA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 2fzjA_ca reduced has been discarded because at least one conformation has only 144 modes

@> WARNING last 10 modes for 144 modes from MaskedGNM 4kebB_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 1u71A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 2w3aA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 1pd9A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 2w3bA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 10 modes for 144 modes from MaskedGNM 3oafA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 2hm9A_ca reduced has been discarded because at least one conformation has only 144 modes


@> WARNING last 12 modes for 144 modes from MaskedGNM 1vj3A_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 12 modes for 144 modes from MaskedGNM 2fzhA_ca reduced has been discarded because at least one conformation has only 144 modes
@> WARNING last 6 modes for 144 modes from MaskedGNM 3td8A_ca reduced has been discarded because at least one conformation has only 144 modes
@> all GNM modes were calculated for each of the 704 conformations in 12.90s.
@> 144 modes across 704 modesets were matched in 3.25s.


In [9]:
IDs = []
with open('./ensemble.txt', 'r') as rf:
    # Split the text into lines and iterate over each line
    for line in rf.readlines():
        if line.startswith('@> WARNING'):
            # Extract the number of discarded modes and the protein structure ID
            parts = line.split()
            for each in parts:
                if "_ca" in each:
                    IDs.append(each.split('_')[0])

# IDs.append('6m18B')
# IDs.append('6m18D')
print (len(IDs))

703


In [10]:
len(ids)

955

# Read dali

In [5]:
# Initialize an empty dictionary to store the PDB IDs and RMSD values
pdb_rmsd_dict = {}

# Open the file and extract the information
with open('1ra3A_dali.txt', 'r') as file:
    # Read the file line by line
    isStart = False
    for line in file:
        # print(line)
        if "# No:  Chain   Z    rmsd lali nres  %id PDB  Description" in line:
            isStart = True
        if "# Structural equivalences" in line:
            isStart = False
        
        # Check if the line starts with a number followed by a colon, which indicates the start of the relevant data
        if isStart and line.strip() != '':
            # Split the line into columns based on whitespace
            columns = line.split()
            
            # Extract the PDB ID and chain, and format it by removing the dash and making it uppercase
            pdb_chain = columns[1].replace('-', '').upper()
            
            # Extract the RMSD value
            rmsd = columns[3]
            length = columns[4]
            
            # Add the information to the dictionary
            pdb_rmsd_dict[pdb_chain] = [rmsd, length]

# Print the dictionary to check
for key, value in pdb_rmsd_dict.items():
    print(f"{key}: {value}")

NO:: ['Z', 'rmsd']
1RA3A: ['0.0', '160']
1RA2A: ['0.2', '160']
1RA8A: ['0.2', '160']
5UIPA: ['0.3', '160']
4X5FA: ['0.2', '160']
4X5FB: ['0.2', '160']
4X5GA: ['0.2', '160']
4X5GB: ['0.2', '160']
5UIPB: ['0.4', '160']
1RA9A: ['0.3', '159']
1RA1A: ['0.3', '159']
4QLEA: ['0.4', '160']
5UIOA: ['0.5', '160']
1RB3A: ['0.5', '160']
1RB3B: ['0.5', '160']
3DRCB: ['0.5', '160']
1DYIB: ['0.4', '160']
1RB2A: ['0.4', '160']
5UIIA: ['0.3', '158']
4QLGA: ['0.5', '160']
1DHIB: ['0.5', '160']
5EAJB: ['0.5', '160']
2DRCB: ['0.5', '160']
1DYHB: ['0.4', '160']
5UIOC: ['0.6', '160']
1JOLB: ['0.5', '160']
4DFRB: ['0.5', '160']
1DRAB: ['0.5', '160']
5UJXB: ['0.5', '160']
1RB2B: ['0.4', '160']
5E8QB: ['0.5', '160']
4FHBA: ['0.5', '160']
1DYJB: ['0.5', '160']
4X5HA: ['0.5', '160']
1DRBB: ['0.5', '160']
1DHJB: ['0.5', '160']
5UIOE: ['0.6', '160']
4QLEB: ['0.6', '160']
1DDSB: ['0.5', '160']
1RD7A: ['0.5', '160']
4QLGB: ['0.6', '160']
1RE7B: ['0.5', '160']
1RD7B: ['0.5', '160']
1RE7A: ['0.5', '160']
8DAIA: ['0.8'

In [6]:
len(pdb_rmsd_dict)

1218

In [7]:
FinalRmsds = []
length = 160
allData = []

for item in ids:
    currLength = pdb_rmsd_dict[item.upper()][1]
    currRMSD = pdb_rmsd_dict[item.upper()][0]
    if float(currRMSD) <= 2:
        if int(currLength) <= 192 and int(currLength) >= 128: 
            FinalRmsds.append(pdb_rmsd_dict[item.upper()][0])
            allData.append([item.lower(), pdb_rmsd_dict[item.upper()][0]])

NameError: name 'ids' is not defined

In [35]:
FinalRmsds

['0.0',
 '0.2',
 '0.2',
 '0.2',
 '0.2',
 '0.2',
 '0.3',
 '0.2',
 '0.4',
 '0.3',
 '0.4',
 '0.3',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '0.4',
 '0.4',
 '0.3',
 '0.5',
 '0.4',
 '0.5',
 '0.5',
 '0.6',
 '0.4',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '0.6',
 '0.5',
 '0.6',
 '0.5',
 '0.5',
 '0.6',
 '0.5',
 '0.5',
 '0.8',
 '0.8',
 '0.8',
 '0.8',
 '0.9',
 '0.8',
 '0.9',
 '0.8',
 '0.9',
 '0.8',
 '0.9',
 '0.9',
 '0.8',
 '0.8',
 '0.8',
 '0.8',
 '0.8',
 '0.8',
 '0.8',
 '0.8',
 '0.9',
 '0.8',
 '0.9',
 '0.8',
 '0.9',
 '0.8',
 '0.9',
 '0.7',
 '0.9',
 '0.8',
 '0.9',
 '0.9',
 '0.5',
 '0.8',
 '0.8',
 '0.8',
 '0.9',
 '0.6',
 '0.8',
 '0.5',
 '0.8',
 '0.8',
 '0.8',
 '0.8',
 '0.9',
 '0.8',
 '0.6',
 '0.9',
 '0.8',
 '0.7',
 '0.9',
 '0.7',
 '0.8',
 '0.9',
 '0.9',
 '0.8',
 '0.5',
 '0.8',
 '0.8',
 '0.9',
 '0.8',
 '0.8',
 '0.9',
 '0.7',
 '0.8',
 '0.8',
 '0.9',
 '0.9',
 '0.8',
 '0.8',
 '0.8',
 '0.8',
 '0.8',
 '0.7',
 '0.8',
 '0.7',
 '0.8',
 '0.7',
 '0.8',
 '0.8',
 '0.8',


In [36]:
with open('similarEnsemble.txt', 'w') as wf:
    for rmsd in FinalRmsds:
        wf.write(rmsd + '\n')